# Mathjax
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>


<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>


**Mathjax config Source**:
[fabriziomusacchio](https://www.fabriziomusacchio.com/blog/2021-08-10-How_to_use_LaTeX_in_Markdown/)





### Power Conversion (PC) – Actuators

| **Metric**                               | **Unit**      | **Description**                                                                   |
|------------------------------------------|---------------|-----------------------------------------------------------------------------------|
| Conversion Efficiency (η_PC)             | %| Ratio of mechanical output energy to electrical input energy.                     |
| Actuation Force                          | N   | Mechanical force produced by the actuator.                                        |
| Displacement / Stroke                    | µm       | Amount of movement generated per actuation cycle.                                |
| Resonant Frequency                       | Hz    | Frequency at which the actuator operates most efficiently.                        |
| Response Time / Rise Time                | ms      | Time taken for the actuator to reach full displacement after activation.          |
| Voltage-to-Displacement Sensitivity      | µm/V          | Displacement achieved per unit of applied voltage.                               |





### Imports

In [562]:
import sympy
from sympy import symbols, Eq, sqrt, pi, latex, solveset, Rational, N
from IPython.display import display, Markdown, Math
import sympy.physics.units as u
from sympy.physics.units import milli, micro, centi, giga, mega, kilo, nano, pico
from sympy.physics.units import convert_to
from math import log10,floor



In [563]:
class num_w_sym:
    def __init__(self, val, sym_str):
        self.val = val
        self.sym = symbols(sym_str)

class eq_as_num_sym:
    def __init__(self, name, func):
        """
        :param name: A string name for the equation.
        :param func: A lambda that accepts a string (either 'sym' or 'val')
                     and returns the desired result.
        """
        self.name = name
        # Evaluate the expression twice: once for symbolic and once for numeric.
        self.sym = Eq(symbols(self.name), func('sym')) 
        self.val = Eq(symbols(self.name), func('val')) 


### Materials

In [564]:

# Materials
class piezo_crystal:
    def __init__(self, name, E, nu, E_c, rho, d_31, d_33, h, l, w):
        """
        Initialize a piezoelectric crystal material.
        :param name: Name of the material
        :param abbrev: Abbreviation of the material name
        :param E: Young's modulus [N/m^2] or [Pa]
        :param J: Intinsic Compliance [m^2/N] or [1/Pa]
        :param nu: Poisson's ratio []
        :param G: Shear modulus [N/m^2] or [Pa]
        :param K: Bulk modulus [N/m^2] or [Pa]
        :param E_c: Coercive Field [V/m]
        :param nu: Poisson's ratio []
        :param rho: Density [kg/m^3]
        :param d31: Piezoelectric coefficient d31 [m/V]
        :param d33: Piezoelectric coefficient d33 [m/V]
        :param h: Height [m]
        :param l: Length [m]
        :param w: Width [m]
        """
        self.name = name
        self.E = num_w_sym(E, f'E^{self.name}')
        self.nu = num_w_sym(nu, f'nu^{self.name}') 
        #self.G = self.E/(2*(1+self.nu))
        #self.K = self.E/(3*(1-2*self.nu))
        self.E_c = self.E_c = num_w_sym(E_c, f'E^c_{self.name}')  
        self.rho = self.tho = num_w_sym(rho, f'rho_{self.name}')  
        self.d_31 = self.d_31 = num_w_sym(d_31, f'd^31_{self.name}')  
        self.d_33 = self.d_33 = num_w_sym(d_33, f'd^33_{self.name}')  
        self.h = self.h = num_w_sym(h, f'h_{self.name}') 
        self.l = self.l = num_w_sym(l, f'l_{self.name}') 
        self.w = self.w = num_w_sym(w, f'w_{self.name}') 


    
class elastic_layer:
    def __init__(self, name, E, nu, rho, h, l, w):
        """
        Initialize an elastic layer material.
        :param name: Name of the material
        :param abbrev: Abbreviation of the material name
        :param E: Young's modulus [N/m^2] or [Pa]
        :param s: elastic Compliance [m^2/N] or [1/Pa]
        :param nu: Poisson's ratio []
        :param rho: Density [kg/m^3]
        :param h: Height [m]
        :param l: Length [m]
        :param w: Width [m]
        """
        self.name = name
        self.E = num_w_sym(E, f'E_{self.name}')
        self.nu = num_w_sym(nu, f'nu_{self.name}') 
        self.rho = num_w_sym(rho, f'rho_{self.name}') 
        self.h = num_w_sym(h, f'h_{self.name}') 
        self.l = num_w_sym(l, f'l_{self.name}') 
        self.w = num_w_sym(w, f'w_{self.name}') 


In [565]:
# Create objects for PZT_5A and steel
p = piezo_crystal(
    name='PZT5A',
    E=63 * 10e6 * u.newton / (u.meter**2),
    E_c= 12 * ((1e3 * u.volt) / (1e-2 * u.meter)),
    nu=0.31,
    rho=7750 * (u.kg / (u.meter**3)),
    d_31=-175 * ((1e-12 * u.meter )/ u.volt),
    d_33=400 * (1e-12 * u.meter / u.volt),
    h=10 *(1e-6 * u.meter),
    l=40 * (1e-6 * u.meter),
    w=100 *(1e-6 * u.meter),
)

s = elastic_layer(
    name='Steel',
    E=193* 10e6 *u.newton / (u.meter**2),
    nu=0.3,
    rho=7872 * (u.kg / (u.meter**3)),
    h=10 *(1e-6* u.meter),
    l=40 * (1e-6* u.meter),
    w=100 *(1e-6* u.meter),
)

In [566]:

# Verifying correct calculation of units
V_max = p.E_c.val * p.h.val / u.volt
V_max.n(3)

12.0

### Basic piezoelectric Actuation equations (latex)


<p align="center">
  <img src="images/piezo_basic_sitti.png" style="width: 70%; height: auto;">
</p>
Source: 978-0-262-03643-6 Page 100 

https://archive.org/details/mobilemicrorobot0000sitt/page/100/


$
\begin{align}
    \delta &= d_{33}V \\
    F_b &= lw \cdot E \cdot \delta \\
    K_m &= \frac{F_b}{\delta} = \frac{lw \cdot E}{h} \\
    f_r &= \frac{N_3^D}{h}
\end{align}
$



In [567]:

# Define symbols
V = symbols('V')
d_33, d_31 = symbols('d_{33} d_{31}')
delta_31, delta_33 = symbols('delta_31 delta_33')
l, w, h = symbols('l w h')
E_33, E_11 = symbols('E_{33} E_{11}')
N_3, N_1 = symbols('N_3 N_1')
K_m = symbols('K_m')
F_b = symbols('F_b')
f_r = symbols('f_r')


# Define equations
ax_displacement = Eq(delta_33, d_33 * V)
ax_Fb = Eq(F_b, (l * w / h) * E_33 * d_33 * V)
ax_coupling = Eq(K_m, (l * w / h) * E_33)
ax_resonance = Eq(f_r, N_3 / h)


tran_displacement = Eq(delta_31, - (w / h) * d_31 * V)
tran_Fb = Eq(F_b, - l * E_11 * d_31 * V) # Blocking force. Maximum Force the Actuator can exert when its displacement is blocked. Aka the Force that can overcome fluid drag and friction
tran_coupling = Eq(K_m, (h * l / w) * E_11) # Electromechanical coupling factor. Higher is better. Max value is 1
tran_resonance = Eq(f_r, N_1 / w)

# Display equations
display(Markdown('Axial Mode'))
for eq in [ax_displacement, ax_Fb, ax_coupling, ax_resonance]:
    display(Math(latex(eq)))
display(Markdown('Transversal Mode'))
for eq in [tran_displacement, tran_Fb, tran_coupling, tran_resonance]:
    display(Math(latex(eq)))

Axial Mode

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Transversal Mode

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Basic piezoelectric Actuation equations (latex)


<p align="center">
  <img src="images/piezo_cantilever_sitti_2.png" style="width: 70%; height: auto;">
</p>

Source: 10.1109/ROBOT.2001.933216 

https://ieeexplore.ieee.org/document/933216


$
\begin{align}
\end{align}
$


In [579]:
R_E = eq_as_num_sym('R_E', lambda attr: getattr(s.E, attr) / getattr(p.E, attr))



display(Eq(R_E.sym,R_E.val.n(3)))

display(R_E.sym)
display(R_E.val.n(3))

out = solveset(R_E.val)
print(out)

out = list(solveset( R_E.val))[0].n(5)
print(out)




Eq(Eq(R_E, E_Steel/E^PZT5A), Eq(R_E, 3.06))

Eq(R_E, E_Steel/E^PZT5A)

Eq(R_E, 3.06)

{3.06349206349206}
3.0635


In [581]:
R_h = eq_as_num_sym('R_h', lambda attr: getattr(s.h, attr) / getattr(p.h, attr))

display(Eq(R_h.sym,R_h.val.n(3)))

out = list(solveset(R_h.val))[0].n(3)
print(out)





Eq(Eq(R_h, h_Steel/h_PZT5A), Eq(R_h, 1.0))

1.00


In [584]:
R_rho = eq_as_num_sym('R_rho',lambda attr: getattr(s.rho, attr) / getattr(s.rho, attr))

display(Eq(R_rho.sym,R_rho.val.n(3)))

out = list(solveset( R_rho.val))[0].n(5)
print(out)

Eq(Eq(R_E, E_Steel/E^PZT5A), Eq(R_E, 3.06))

3.0635
